In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import keras as ks
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPool2D, Conv2DTranspose, BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
%load_ext Cython

In [ ]:
%%cython
cimport cython
import numpy as np

@cython.cdivision(True)
@cython.boundscheck(False)
@cython.nonecheck(False)
@cython.wraparound(False)
cdef int calc_neighs(unsigned char[:, :] field, int i, int j, int n, int k):
    cdef:
        int neighs = 0;
        int i_min = i - 1;
        int i_pl = i + 1;
        int j_min = j - 1;
        int j_pl = j + 1;
    neighs = 0
    if i_min >= 0:
        if j_min >= 0:
            neighs += field[i_min, j_min]
        neighs += field[i_min, j]
        if j_pl < k:
            neighs += field[i_min, j_pl]
    if j_min >= 0:
        neighs += field[i, j_min]
    if j_pl < k:
        neighs += field[i, j_pl]
    if i_pl < n:
        if j_min >= 0:
            neighs += field[i_pl, j_min]
        neighs += field[i_pl, j]
        if j_pl < k:
            neighs += field[i_pl, j_pl]
    return neighs

@cython.cdivision(True)
@cython.boundscheck(False)
@cython.nonecheck(False)
@cython.wraparound(False)
cpdef make_move(unsigned char[:, :] field, int moves):
    cdef:
        int _, i, j, neighs;
        int n, k;
        int switch = 0;
        unsigned char[:, :] cur_field;
        unsigned char[:, :] next_field;
    cur_field = np.copy(field)
    next_field = np.zeros_like(field, 'uint8')
    n = field.shape[0]
    k = field.shape[1]
    for _ in range(moves):
        if switch == 0:
            for i in range(n):
                for j in range(k):
                    neighs = calc_neighs(cur_field, i, j, n, k)
                    if cur_field[i, j] and neighs == 2:
                        next_field[i, j] = 1
                    elif neighs == 3:
                        next_field[i, j] = 1
                    else:
                        next_field[i, j] = 0
        else:
            for i in range(n):
                for j in range(k):
                    neighs = calc_neighs(next_field, i, j, n, k)
                    if next_field[i, j] and neighs == 2:
                        cur_field[i, j] = 1
                    elif neighs == 3:
                        cur_field[i, j] = 1
                    else:
                        cur_field[i, j] = 0
        switch = (switch + 1) % 2
    return np.array(next_field if switch else cur_field)

In [ ]:
NROW, NCOL = 20, 20

def generate_samples(delta=1, n=32):
    """
    Generate batch of samples
    
    @return: (end_frames, start_frames)
    """
    batch = np.split(np.random.binomial(1, 0.5, (NROW * n, NCOL)).astype('uint8'), n)
    Yy = [life.make_move(state, 5) for state in batch]
    Xx = [life.make_move(state, 1) for state in Yy]
    Y = np.array([y.ravel() for y in Yy])
    X = np.array([x.ravel() for x in Xx])
    return X, Y
    

def data_generator(delta=1, batch_size=32, ravel=True):
    """
    Can be used along with .fit_generator to generate training samples on the fly
    """
    while True:
        batch = np.split(np.random.binomial(1, 0.5, (NROW * batch_size, NCOL)).astype('uint8'), batch_size)
        Yy = [make_move(state, 5) for state in batch]
        Xx = [make_move(state, delta) for state in Yy]

        if ravel:
            Y = np.array([y.ravel() for y in Yy])
            X = np.array([x.ravel() for x in Xx])
            yield X, Y
        else:
            yield np.array(Xx)[:,:, :, np.newaxis], np.array(Yy)[:, :, :, np.newaxis]

In [ ]:
def create_model(n_hidden_convs=2, n_hidden_filters=128, kernel_size=5):
    nn = Sequential()
    nn.add(Conv2D(n_hidden_filters, kernel_size, padding='same', activation='relu', input_shape=(20, 20, 1)))
    nn.add(BatchNormalization())
    for i in range(n_hidden_convs):
        nn.add(Conv2D(n_hidden_filters, kernel_size, padding='same', activation='relu'))
        nn.add(BatchNormalization())
    nn.add(Conv2D(1, kernel_size, padding='same', activation='sigmoid'))
    nn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return nn

def model_v2():
    # This one performs worse then pure Conv2D
    nn = Sequential()
    nn.add(Conv2D(128, 5, padding='same', activation=lrelu, input_shape=(20, 20, 1)))
    nn.add(BatchNormalization())
    nn.add(Conv2D(128, 5, padding='valid', activation=lrelu))
    nn.add(BatchNormalization())
    nn.add(MaxPool2D())
    nn.add(Conv2DTranspose(128, 2, strides=(2, 2), padding='valid', activation=lrelu))
    nn.add(BatchNormalization())
    nn.add(Conv2DTranspose(128, 5, strides=(1, 1), padding='valid', activation=lrelu))
    nn.add(BatchNormalization())
    nn.add(Conv2D(1, 5, padding='same', activation='sigmoid'))
    nn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return nn

In [ ]:
models = []
for delta in range(1, 6):
    model = create_model(n_hidden_convs=6, n_hidden_filters=256)
    es = EarlyStopping(monitor='loss', patience=9, min_delta=0.001)
    model.fit_generator(data_generator(delta=delta, ravel=False), steps_per_epoch=500, epochs=50, verbose=1, callbacks=[es])
    models.append(model)

In [ ]:
train_df = pd.read_csv('../input/train.csv', index_col=0)
test_df = pd.read_csv('../input/test.csv', index_col=0)

In [ ]:
submit_df = pd.DataFrame(index=test_df.index, columns=['start.' + str(_) for _ in range(1, 401)])

In [ ]:
for delta in range(1, 6):
    mod = models[delta-1]
    delta_df = test_df[test_df.delta == delta].iloc[:, 1:].values.reshape(-1, 20, 20, 1)
    submit_df[test_df.delta == delta] = mod.predict(delta_df).reshape(-1, 400).round(0).astype('uint8')

In [ ]:
submit_df.to_csv('cnns_40.csv')